In [2]:
#Importing packages
import pandas as pd
from sqlalchemy import create_engine

print("Libraries imported!")

Libraries imported!


In [3]:
#Create sql engine
engine = create_engine("sqlite:///mta.db")

#Load db using real_sql
all_data = pd.read_sql('SELECT * FROM mta_data;', engine)

#Display first 5 rows
all_data.head()

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS ...
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,03:00:00,REGULAR,7543925,2573549
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,07:00:00,REGULAR,7543931,2573559
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,11:00:00,REGULAR,7543970,2573622
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,15:00:00,REGULAR,7544072,2573700


In [4]:
#We see the first row in the all_data df is the headers, so let's drop it and reset index
all_data = all_data.drop(0, axis = 0).reset_index(drop=True)
all_data

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,03:00:00,REGULAR,7543925,2573549
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,07:00:00,REGULAR,7543931,2573559
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,11:00:00,REGULAR,7543970,2573622
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,15:00:00,REGULAR,7544072,2573700
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/13/2021,19:00:00,REGULAR,7544219,2573729
...,...,...,...,...,...,...,...,...,...,...,...
3353756,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,12/04/2020,04:00:00,REGULAR,5554,541
3353757,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,12/04/2020,08:00:00,REGULAR,5554,541
3353758,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,12/04/2020,12:00:00,REGULAR,5554,541
3353759,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,12/04/2020,16:00:00,REGULAR,5554,541


In [5]:
all_data.shape

(3353761, 11)

In [6]:
all_data.dtypes

CA          object
UNIT        object
SCP         object
STATION     object
LINENAME    object
DIVISION    object
DATE        object
TIME        object
DESC        object
ENTRIES     object
EXITS       object
dtype: object

In [7]:
all_data.columns

Index(['CA', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')

In [8]:
all_data['DATE'] = pd.to_datetime(all_data['DATE'])
all_data.head()

,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-03-13,03:00:00,REGULAR,7543925,2573549
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-03-13,07:00:00,REGULAR,7543931,2573559
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-03-13,11:00:00,REGULAR,7543970,2573622
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-03-13,15:00:00,REGULAR,7544072,2573700
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,2021-03-13,19:00:00,REGULAR,7544219,2573729


In [51]:
all_data.describe()

<ipython-input-51-14a90e0e01c9>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  all_data.describe()


,CA,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
count,3353761,3353761,3353761,3353761,3353761,3353761,3353761,3353761,3353761,3353761.0,3353761.0
unique,750,469,228,378,114,6,112,61293,2,2107269.0,2008577.0
top,PTH22,R549,00-00-00,34 ST-PENN STA,1,IRT,2020-12-19 00:00:00,03:00:00,REGULAR,0.0,0.0
freq,28760,45896,311494,68287,410308,1202140,31271,270084,3339952,47039.0,133680.0
first,NaN,NaN,NaN,NaN,NaN,NaN,2020-11-28 00:00:00,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-19 00:00:00,NaN,NaN,NaN,NaN


In [9]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3353761 entries, 0 to 3353760
Data columns (total 11 columns):
 #   Column    Dtype         
---  ------    -----         
 0   CA        object        
 1   UNIT      object        
 2   SCP       object        
 3   STATION   object        
 4   LINENAME  object        
 5   DIVISION  object        
 6   DATE      datetime64[ns]
 7   TIME      object        
 8   DESC      object        
 9   ENTRIES   object        
 10  EXITS     object        
dtypes: datetime64[ns](1), object(10)
memory usage: 281.5+ MB
